In [35]:
import sacrebleu
import json
from transformers import AutoTokenizer
import evaluate

In [36]:
sb = evaluate.load('sacrebleu')

In [12]:
tokenizer = AutoTokenizer.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased')

In [2]:
data = json.load(open('/mnt/c/Users/Admin/Downloads/work/coco/caption_human_thai_train2017.json'))

In [3]:
data.keys()

dict_keys(['info', 'licenses', 'images', 'annotations'])

In [47]:
d2 = json.load(open('/mnt/c/Users/Admin/Downloads/work/coco/caption_human_thai_val2017.json'))
d2.keys()

dict_keys(['info', 'licenses', 'images', 'annotations'])

In [57]:
data['annotations'].keys()

dict_keys(['image_id', 'caption_thai'])

# Train: Human -> Human

In [99]:
captions = {}
for id, caption in zip(data['annotations']['image_id'], data['annotations']['caption_thai']):
    if id not in captions:
        captions[id] = []
    captions[id].append(caption)

In [100]:
print(list(captions.values())[3])
k = 4
gt = []
pd = []
for caps in captions.values():
    if len(caps) < 2:
        continue
    pd.append(' '.join([tokenizer.decode(x) for x in tokenizer(caps.pop(0))['input_ids'][1:-1]]).replace('  ', ' '))
    labs = []
    tks = tokenizer(caps)['input_ids']
    for tk in tks:
        labs.append(' '.join([tokenizer.decode(x) for x in tk[1:-1]]).replace(' ', ' '))
    if len(labs) < k:
        for i in range(k-len(labs)):
            labs.append('')
    gt.append(labs[:k])
print(pd[:3])
print(gt[:3])

['จานสีขาวจานนึงขบวนการนั้นมี โดนัทสอดไส้ทูน่าอยู่ในจาน', 'ภาพของเบอร์เกอร์โดนัทที่หั่นครึ่งชิ้นอยู่ในจานบนโต๊ะ', 'ขนมปังที่เป็นโดนัตวางอยู่บนจานสีขาวที่อยู่บนโต๊ะไม้']
['มี แม่ คนหนึ่ง สวมเสื้อ แขนยาว ที่กําลัง เล่นกับ ลูก อยู่ บล สวน ห ย่อม', ' สีน้ําตาล ตัวเล็ก กําลัง กินอาหาร อยู่บน จาน กระดาษ สีขาว ข้าง แก้ว น้ํา', ' ไป สัญญาณ จราจร 3 อัน ติด อยู่บน เสาไฟ สีดํา แสดง สัญญาณไฟ สีแดง']
[['สาว คนนึง กําลัง พ า เด็ก มา นั่งเล่น อยู่ภายใน สนามหญ้า พร้อมกับ สุนัข', '', '', ''], [' นก น้อย ตัวหนึ่ง กําลัง จิก กิน เศษ อาหารที่ วาง ทิ้งไว้ บนโต๊ะ', ' บนโต๊ะ อาหาร  นก น้อย ตัวหนึ่ง กําลัง ก้มหน้า กินอะไร บางอย่าง', '', ''], [' สัญญาณไฟ จราจร ที่ถูก ติดตั้ง อยู่กับ เสาไฟ อยู่ริมถนน', ' สัญญาณไฟ จราจร ที่ได้ ติด อยู่บน เสา ต้น หนึ่งที่ ได้ ตั้งอยู่ใน ริมถนน', '', '']]


In [101]:
sacrebleu.corpus_bleu(
    hypotheses = pd,
    references = gt
)

BLEU = 16.43 59.2/46.7/19.5/1.4 (BP = 1.000 ratio = 1.361 hyp_len = 49 ref_len = 36)

In [102]:
print(sb.compute(predictions=pd, references=gt))

{'score': 5.575309916166869, 'counts': [335425, 94052, 29020, 9256], 'totals': [1095840, 1013361, 930882, 848403], 'precisions': [30.608939261206014, 9.281193967401547, 3.1174735358509458, 1.0909909559490008], 'bp': 1.0, 'sys_len': 1095840, 'ref_len': 987326}


In [62]:
gg = set()
for g in gt:
    gg.add(len(g))
gg

{2}

# val: Model -> Human

In [72]:
d3 = json.load(open('/mnt/c/Users/Admin/Downloads/capgen_api-main/preds.json'))
labels = {}
predicts = {}
ids = []
for d in d2['annotations']:
    idx = d['image_id']
    if idx not in ids:
        predicts[idx] = d3[len(ids)]
        ids.append(idx)
    if idx not in labels:
        labels[idx] = []
    labels[idx].append(d['caption_thai'])


In [106]:
k = 4
gt = []
pd = []
for key in labels:
    caps = labels[key]
    if len(caps) < 2:
        continue
    pd.append(
        ' '.join([tokenizer.decode(x) for x in tokenizer(predicts[key])['input_ids'][1:-1]]).replace('  ', ' ')
    )
    labs = []
    tks = tokenizer(caps)['input_ids']
    for tk in tks:
        labs.append(
            ' '.join([tokenizer.decode(x) for x in tk[1:-1]]).replace(' ', ' ')
        )
    if len(labs) < k:
        for i in range(k-len(labs)):
            labs.append('')
    gt.append(labs[:k])
print(pd[:3])
print(gt[:3])

['ผู้ชาย สวมเสื้อ แขนยาว สีดํา สวม กางเกง สีน้ําเงิน กําลัง ยืน อยู่บน สกี หิมะ', ' แมว สีดํา น้ําตาล 2 ตัว กําลัง ยืน อยู่ข้าง จาน อาหาร สีขาว ใส่ สุนัข สีน้ําตาล ดํา และ สุนัข', ' สนาม เบสบอล มีนัก เบสบอล กําลัง แข่งขันกัน อยู่ และมี ผู้ชม กําลัง ดู อยู่']
[['คน สวมเสื้อ แขนยาว สีดํา สวม ก างเกงขายาว สีน้ําเงิน กําลัง ยืน อยู่บน สกี หิมะ', 'ผู้ชาย สองคน ที่กําลัง เดิน อยู่ บนภูเขา หิมะ ของ ลาน สกี หิมะ', ' บน เทือกเขา สูง ที่มี หิมะ ปกคลุม สีขาว หนา  มีนัก เล่น สกี สองคน อยู่ ตรงนั้น', ''], [' หมา ตัว สีดํา น้ําตาล กําลัง มอง มา ที่ แมว ตัว สีเทา นั่งอยู่ บนโต๊ะ', ' แมว ตัว สีเทา ที่ นั่งอยู่ บนโต๊ะ มาแล้ว มี สุนัข ตัว สีดํา อยู่ นอกบ้าน', ' แมว สีเทา  1  ตัว นั่ง อยู่ข้างๆ กระถาง ต้นไม้ บนโต๊ะ  ด้านหลัง มี สุนัข  1  ตัว ยืนอยู่ ด้านนอก บ้าน', ''], ['ผู้ชาย ใส่เสื้อ สีขาว กําลัง ตั้ง ท่า ตี ลูก เบสบอล อยู่ใน สนาม', ' นักกีฬา เบสบอล ที่ทําการ แข่งขันกีฬ า อยู่ใน สนาม มี สนามหญ้า อยู่ บริเวณรอบ รอบ', ' ด้านหน้า มีคน หลายคน มี ผู้ชาย ยืน เล่น เบสบอล อยู่ใน สนาม  ด้านหลัง มีคน นั่ง อยู่บ

In [107]:
sacrebleu.corpus_bleu(
    hypotheses = pd,
    references = gt
)

BLEU = 34.52 76.9/54.2/27.3/12.5 (BP = 1.000 ratio = 1.268 hyp_len = 52 ref_len = 41)

In [108]:
print(sb.compute(predictions=pd, references=gt))

{'score': 12.022351202904273, 'counts': [19168, 6810, 2582, 998], 'totals': [40910, 37519, 34128, 30737], 'precisions': [46.854069909557566, 18.15080359284629, 7.565635255508673, 3.246901128932557], 'bp': 1.0, 'sys_len': 40910, 'ref_len': 39994}


# val: Human -> Human

In [112]:
d3 = json.load(open('/mnt/c/Users/Admin/Downloads/capgen_api-main/preds.json'))
labels = {}
predicts = {}
ids = []
for d in d2['annotations']:
    idx = d['image_id']
    if idx not in ids:
        predicts[idx] = d3[len(ids)]
        ids.append(idx)
    if idx not in labels:
        labels[idx] = []
    labels[idx].append(d['caption_thai'])

In [113]:
print(list(labels.values())[3])
k = 2
gt = []
pd = []
for caps in labels.values():
    if len(caps) < 2:
        continue
    pd.append(' '.join([tokenizer.decode(x) for x in tokenizer(caps.pop(0))['input_ids'][1:-1]]).replace('  ', ' '))
    labs = []
    tks = tokenizer(caps)['input_ids']
    for tk in tks:
        labs.append(' '.join([tokenizer.decode(x) for x in tk[1:-1]]).replace(' ', ' '))
    if len(labs) < k:
        for i in range(k-len(labs)):
            labs.append('')
    gt.append(labs[:k])
print(pd[:3])
print(gt[:3])

['คนสวมเสื้อลายสก๊อตสีฟ้าส่งกางเกงยีนส์ขายาวนั่งทำว่าวบนพื้นกระเบื้อง']
['คน สวมเสื้อ แขนยาว สีดํา สวม ก างเกงขายาว สีน้ําเงิน กําลัง ยืน อยู่บน สกี หิมะ', ' หมา ตัว สีดํา น้ําตาล กําลัง มอง มา ที่ แมว ตัว สีเทา นั่งอยู่ บนโต๊ะ', 'ผู้ชาย ใส่เสื้อ สีขาว กําลัง ตั้ง ท่า ตี ลูก เบสบอล อยู่ใน สนาม']
[['ผู้ชาย สองคน ที่กําลัง เดิน อยู่ บนภูเขา หิมะ ของ ลาน สกี หิมะ', ' บน เทือกเขา สูง ที่มี หิมะ ปกคลุม สีขาว หนา  มีนัก เล่น สกี สองคน อยู่ ตรงนั้น'], [' แมว ตัว สีเทา ที่ นั่งอยู่ บนโต๊ะ มาแล้ว มี สุนัข ตัว สีดํา อยู่ นอกบ้าน', ' แมว สีเทา  1  ตัว นั่ง อยู่ข้างๆ กระถาง ต้นไม้ บนโต๊ะ  ด้านหลัง มี สุนัข  1  ตัว ยืนอยู่ ด้านนอก บ้าน'], [' นักกีฬา เบสบอล ที่ทําการ แข่งขันกีฬ า อยู่ใน สนาม มี สนามหญ้า อยู่ บริเวณรอบ รอบ', ' ด้านหน้า มีคน หลายคน มี ผู้ชาย ยืน เล่น เบสบอล อยู่ใน สนาม  ด้านหลัง มีคน นั่ง อยู่บน อั ฒ จันทร์ หลายคน']]


In [114]:
sacrebleu.corpus_bleu(
    hypotheses = pd,
    references = gt
)

BLEU = 23.46 100.0/66.7/18.2/2.5 (BP = 1.000 ratio = 1.000 hyp_len = 26 ref_len = 26)

In [116]:
print(sb.compute(predictions=pd, references=gt))

{'score': 7.016280146118383, 'counts': [14709, 4469, 1416, 439], 'totals': [41321, 37930, 34539, 31148], 'precisions': [35.59691198180102, 11.782230424466121, 4.099713367497611, 1.4094002825221523], 'bp': 1.0, 'sys_len': 41321, 'ref_len': 40900}


In [ ]:
a = 1

# val: MTL -> Human

In [117]:
d4 = json.load(open('/mnt/c/Users/Admin/Downloads/work/coco/caption_thai_val2017.json'))
d4.keys()

dict_keys(['info', 'licenses', 'images', 'annotations'])

In [119]:
d4['annotations'][0]

{'image_id': 179765,
 'id': 38,
 'caption': 'A black Honda motorcycle parked in front of a garage.',
 'caption_thai': 'รถจักรยานยนต์ฮอนด้าสีดำจอดอยู่หน้าโรงรถ'}

In [122]:
labels = {}
predicts = {}
ids = []
for d in d2['annotations']:
    idx = d['image_id']
    if idx not in labels:
        labels[idx] = []
    labels[idx].append(d['caption_thai'])
for d in d4['annotations']:
    idx = d['image_id']
    if idx not in predicts:
        predicts[idx] = []
    predicts[idx].append(d['caption_thai'])


In [124]:
list(labels.keys())[0]

'205333'

In [125]:
list(predicts.keys())[0]

179765

In [130]:
k = 2
gt = []
pd = []
for key in labels:
    caps = labels[key]
    if len(caps) < 2:
        continue
    pd.append(
        ' '.join([tokenizer.decode(x) for x in tokenizer(predicts[int(key)][0])['input_ids'][1:-1]]).replace('  ', ' ')
    )
    labs = []
    tks = tokenizer(caps)['input_ids']
    for tk in tks:
        labs.append(
            ' '.join([tokenizer.decode(x) for x in tk[1:-1]]).replace(' ', ' ')
        )
    if len(labs) < k:
        for i in range(k-len(labs)):
            labs.append('')
    gt.append(labs[:k])
print(pd[:3])
print(gt[:3])

['นัก สกี คู่ หนึ่ง อยู่ ด้านบน ของ สกี ที่มี เมฆ และ ภูเขา ขนาดใหญ่ อยู่ ด้านหลัง', ' แมว นั่งอยู่ บนโต๊ะ พร้อม สุนัข ข้างนอก', ' ชายคนหนึ่ง ขว้าง เบสบอล จาก กอง บน สนาม']
[['คน สวมเสื้อ แขนยาว สีดํา สวม ก างเกงขายาว สีน้ําเงิน กําลัง ยืน อยู่บน สกี หิมะ', 'ผู้ชาย สองคน ที่กําลัง เดิน อยู่ บนภูเขา หิมะ ของ ลาน สกี หิมะ'], [' หมา ตัว สีดํา น้ําตาล กําลัง มอง มา ที่ แมว ตัว สีเทา นั่งอยู่ บนโต๊ะ', ' แมว ตัว สีเทา ที่ นั่งอยู่ บนโต๊ะ มาแล้ว มี สุนัข ตัว สีดํา อยู่ นอกบ้าน'], ['ผู้ชาย ใส่เสื้อ สีขาว กําลัง ตั้ง ท่า ตี ลูก เบสบอล อยู่ใน สนาม', ' นักกีฬา เบสบอล ที่ทําการ แข่งขันกีฬ า อยู่ใน สนาม มี สนามหญ้า อยู่ บริเวณรอบ รอบ']]


In [131]:
sacrebleu.corpus_bleu(
    hypotheses = pd,
    references = gt
)

BLEU = 11.05 100.0/36.8/2.9/1.7 (BP = 0.953 ratio = 0.955 hyp_len = 21 ref_len = 22)

In [132]:
print(sb.compute(predictions=pd, references=gt))

{'score': 2.853767518743874, 'counts': [9488, 1979, 400, 77], 'totals': [26930, 23539, 20148, 16757], 'precisions': [35.23208317861121, 8.407324015463699, 1.9853087155052611, 0.45950945873366356], 'bp': 0.703864298497585, 'sys_len': 26930, 'ref_len': 36387}
